In [423]:
import numpy as np
import pandas as pd
import trimesh as tm
from matplotlib import pyplot as plt

np.set_printoptions(precision=3, suppress=True)

In [424]:
def load_template(file_path):
        faces = []
        normals = []
        i = 0
        with open(file_path) as file:
            for line in file:
                if line[0] == "f":
                    line_split = line[1:].split()
                    line_split = np.array([i.split("//") for i in line_split])
                    face_temp = list(map(int, line_split[:, 0]))
                    #normal_temp = list(map(int, line_split[:, 1]))
                    faces.append(face_temp)
                    #normals.append(normal_temp)
                    i += 1
        return np.array(faces), np.array(normals)

def load_mesh(file_path):
    vertex = []
    with open(file_path, "r") as f:
        for line in f:
            if line[0] == '#':
                continue
            elif "v " in line:
                line.replace('\n', ' ')
                tmp = list(map(float, line[1:].split()))
                vertex.append(tmp)
            else:
                break
    vertex = np.array(vertex)
    return vertex
vertices = load_mesh('../curves-modules/mean_female_pose_1.obj')
faces, _ = load_template('../curves-modules/mean_female_pose_1.obj')

def save_obj(path, pontos, faces=[]):
    with open(path, "w") as file:
        for ponto in pontos:
            file.write("v {} {} {}\n".format(ponto[0], ponto[1], ponto[2]))
        for face in faces:
            file.write("f {} {} {}\n".format(face[0], face[1], face[2]))

def generate_positions(body, coordinates):
    positions = []
    for i in range(0, len(coordinates)):
        x = body[int(coordinates[i][0])] * coordinates[i][3]
        y = body[int(coordinates[i][1])] * coordinates[i][4]
        z = body[int(coordinates[i][2])] * coordinates[i][5]
        p = x + y + z
        positions.append(p)
    return np.array(positions)

In [425]:
vertices = load_mesh('../curves-modules/mean_female_pose_0.obj')
faces, _ = load_template('../curves-modules/mean_female_pose_0.obj')
faces-=1

In [506]:
nplane = np.array([0.0,-1.0,0.0])
dplane = 0.0

In [525]:
def baricentric(vertices, p):
    a = vertices[:,0]
    b = vertices[:,1]
    c = vertices[:,2]
    
    vab = b - a
    vbc = c - b
    vca = a - c
    
    vap = p - a
    vbp = p - b
    vcp = p - c
    n = np.cross(vab,vbc) / np.linalg.norm(np.cross(vab,vbc))
    
    
    ABC = (n * np.cross(vab, vbc)).sum(axis=1) / 2
    ABP = (n * np.cross(vab, vbp)).sum(axis=1) / 2
    BCP = (n * np.cross(vbc, vcp)).sum(axis=1) / 2
    CAP = (n * np.cross(vca, vap)).sum(axis=1) / 2
    
    w = ABP/ABC
    u = BCP/ABC
    v = CAP/ABC
    colisions = np.column_stack([u,v,w])
    valid = colisions.min(axis=1) >= 0
    return p[valid], valid

def intersection(p0, p1):

    vd = p0 - p1
    integer_part = np.dot(p0, nplane)
    print(integer_part)

    integer_part = np.inner(p0, nplane)
    print(integer_part)

    incognita_part = np.inner(nplane, vd)
    t = (dplane + integer_part)/incognita_part
    p = p0 + np.multiply(vd, t[:, np.newaxis])
    return p


def calculate_colision(vertices, faces):

    v = vertices[faces]
    v = v.reshape(v.size//3, 3)
    colision = np.sum((v*nplane) + dplane, axis=1)
    colision = colision.reshape(colision.size//3, 3)
    v = v.reshape(v.size//9,3,3)

    t1 = np.all(colision < 0, axis=1)
    t2 = np.all(colision > 0, axis=1)
    t3 = np.invert(np.any(np.column_stack((t1,t2)), axis=1))

    f = faces[t3]
    colision = colision[t3]
    v = v[t3]
    
    a = np.array([v[:,1]-v[:,0], v[:,2]-v[:,1], v[:,0]-v[:,2]])
    a = a.reshape((a.size//9,3,3))
    
    p = np.vstack([
        intersection(v[:,0], v[:,1]),
        intersection(v[:,0], v[:,2]),
        intersection(v[:,1], v[:,0]),
        intersection(v[:,1], v[:,2]),
        intersection(v[:,2], v[:,0]),
        intersection(v[:,2], v[:,1])
    ])



coordinates = calculate_colision(vertices, faces)
# save_obj('test.obj', generate_positions(vertices, coordinates))

[-0.001  0.017  0.003 -0.001 -0.011  0.008 -0.001  0.016  0.013 -0.004
  0.008 -0.008  0.013 -0.005  0.013 -0.007  0.011 -0.005  0.012 -0.004
  0.003  0.003  0.02  -0.001 -0.003  0.015 -0.019  0.003  0.013 -0.006
  0.015 -0.003  0.016 -0.001  0.017 -0.004 -0.007  0.013 -0.002 -0.001
  0.002  0.02  -0.001  0.011  0.006  0.014  0.015 -0.007 -0.007 -0.011
 -0.007 -0.007  0.013  0.013 -0.011 -0.008 -0.008 -0.007 -0.008 -0.001
 -0.004 -0.002  0.015  0.013  0.006  0.002 -0.004 -0.007  0.013  0.011
 -0.001 -0.001  0.018  0.014  0.015  0.013]
[-0.001  0.017  0.003 -0.001 -0.011  0.008 -0.001  0.016  0.013 -0.004
  0.008 -0.008  0.013 -0.005  0.013 -0.007  0.011 -0.005  0.012 -0.004
  0.003  0.003  0.02  -0.001 -0.003  0.015 -0.019  0.003  0.013 -0.006
  0.015 -0.003  0.016 -0.001  0.017 -0.004 -0.007  0.013 -0.002 -0.001
  0.002  0.02  -0.001  0.011  0.006  0.014  0.015 -0.007 -0.007 -0.011
 -0.007 -0.007  0.013  0.013 -0.011 -0.008 -0.008 -0.007 -0.008 -0.001
 -0.004 -0.002  0.015  0.013  0.0